# 定期預金口座獲得のためのダイレクトマーケティング

## 分析設計

### 1. ターゲットのユーザー像（ペルソナ）

テレマーケティングの結果 `y` を被説明変数とする**ロジスティック回帰分析**をRで実施し、  
オッズ比をもとに成約を左右する要素を特定し、ペルソナの解釈を行う。

### 2. 予測モデルの分析設計

$ROI = 5000 \times 成約数 - 500 \times 架電数$  
の最大化が目的。
ロジスティック回帰とランダムフォレストを両方試したところ  
**ロジスティック回帰**のほうが明らかにAUCが高かったため、後者を採用した。

## ペルソナの分析結果

ロジスティック回帰分析の結果から想定される典型的顧客像は、大きく分けて以下の3つ。
1. **成約歴のある60〜80代**
2. **成約歴のある学生**
3. **10代**

このうち、1. と 2. は連絡手段が携帯電話であることが望ましい。

### ペルソナの分析過程

ペルソナ分析に利用したのは、データセットのうち以下の12種。  
属人的かつ今回のテレマーケティング実施前に明らかな11要素を説明変数とし、  
マーケティング結果 `y` を被説明変数とした。

|変数|意味|被説明変数/説明変数|
|:---|:---|:---|
|y|今回のマーケティング結果|被説明変数|
|age|年齢|説明変数|
|job|職種|説明変数|
|marital|婚姻状況|説明変数|
|education|最終学歴|説明変数|
|default|クレジット支払遅延|説明変数|
|housing|不動産ローンの有無|説明変数|
|loan|個人ローンの有無|説明変数|
|contact|連絡デバイス|説明変数|
|day_of_week|最終連絡曜日|説明変数|
|previous|前回キャンペーン時の接触回数|説明変数|
|poutcome|前回キャンペーンの結果|説明変数|

#### 前処理
**サンプリング**  
予測が目的ではないので学習用とテスト用には分けていない。

**多重共線性への対処**  
個人ローン `loan` と住宅ローン `housing` で、`unknown` の顧客が一致しているため、必ず多重共線性の問題が発生する。  
また、それ以外の説明変数も多重共線性の問題を抱えることが想定されるが、  
問題を発見したときに列ごと除外すると、残しておいてよい変数まで消してしまう。  

そこで、各説明変数は先にダミー変数に置き換えておき、確定している `housing_unknown` をあらかじめ除外、  
残った `yes` と `no` も逆相関のため片方だけあればよいので、`yes` だけを使用した。

**年齢の扱い**  
成約が若年層と高齢者層に偏っており、`age` と `y` に比例関係を仮定できないため、  
連続値ではなく10歳ごとに区切ったカテゴリカル変数に置き換えた。  

結果として、説明変数は44列となった。

#### モデルの構築
1. 44の説明変数をすべて投入したフルモデルを作成
2. z検定で p値 > 0.05 の変数を除外した改善モデルを作成
3. `car::vif()` で多重共線性の問題がないか確認（今回は常に5.0未満のため除外せず）
4. ステップワイズ法 `step()` で変数を増減させて最適なモデルを選択
5. オッズ比を出力

### オッズ比に基づくペルソナの解釈
オッズ比が1.0を超えたものは以下の表のとおり。  

|変数|変数の意味|オッズ比|
|:---|:---|:---|
|poutcome_success|以前のキャンペーン結果_成約|8.41133189|
|age_group_8|年齢層_80代|4.80497216|
|age_group_7|年齢層_70代|4.37320628|
|age_group_6|年齢層_60代|3.09050875|
|age_group_1|年齢層_10代|1.84904789|
|job_student|職業_学生|1.78250620|
|previous|以前のキャンペーン時の接触回数|1.46950908|
|poutcome_nonexistent|以前のキャンペーン結果_なし|1.35041915|
|age_group_2|年齢層_20代|1.31262599|
|education_unknown|最終学歴_不明|1.26241286|
|age_group_5|年齢層_50代|1.19808194|
|marital_single|婚姻状況_独身|1.15987809|

今回はこのうち
- **以前のキャンペーン結果_成約**
- **年齢層_60〜80代**
- **年齢層_10代**
- **職業_学生**

の4つに着目し、それぞれについて個別に `y` を被説明変数とするロジスティック回帰を行なうことで  
より精緻なターゲット顧客像の把握を目指した。

#### 以前に成約した層の場合
**以前のキャンペーンで成約していること自体が重要**。

|変数|変数の意味|オッズ比|
|:---|:---|:---|
|(Intercept)|-|1.3981246|
|previous|以前のキャンペーン時の接触回数|1.3741081|
|day_of_week_mon|最終連絡曜日_月曜|0.5453168|
|education_basic.9y|最終学歴_義務教育|0.5350085|
|default_unknown|クレジットの支払遅延_不明|0.4828141|
|job_unknown|職業_不明|0.2578113|

`previous` のオッズ比は全体のペルソナ分析でのオッズ比とほぼ同等なので  
`poutcome_success` に限って特にプラスに効いている変数はない。  

身元が不確かな相手は避けるべき。

#### 高齢者の場合
**以前のキャンペーンで成約したことのある60歳以上**が有力な顧客像。  
できれば携帯で連絡が取れることが望ましい。

|変数|変数の意味|オッズ比|
|:---|:---|:---|
|poutcome_success|以前のキャンペーン結果_成約|5.7583578|
|day_of_week_tue|最終連絡曜日_火曜|1.5716314|
|education_university.degree|最終学歴_大卒|0.6234444|
|(Intercept)|-|0.6216729|
|contact|連絡手段_固定電話|0.4818810|
|default_unknown|クレジットの支払遅延_不明|0.4008915|
|job_unknown|職業_不明|0.2961794|
|education_basic.9y|最終学歴_義務教育|0.2757232|
|job_blue-collar|職業_現場労働|0.1259056|

`default_unknown`, `job_unknown`, `education_basic`, `job_blue-collar` は  
オッズ比が0.5未満なので、避けるほうが望ましい。

#### 10代の場合
**10代であればほかに条件は問わない**

有意な説明変数は存在しなかった。

#### 学生の場合
**以前のキャンペーンで成約した学生**が有力な顧客像。
できれば携帯電話で連絡が取れることが望ましい。

|変数||オッズ比|
|:---|:---|:---|
|poutcome_success|以前のキャンペーン結果_成約|5.6730590|
|(Intercept)|-|0.4414258|
|contact|連絡デバイス_固定電話|0.4253912|
|default_unknown|クレジットの支払遅延_不明|0.3151173|

`default_unknown` はオッズ比が0.5未満なので、避けたい要素。

## 予測モデル

ロジスティック回帰の結果、

**ROI = 171万8000円**


予測モデル構築に利用したのは、データセットのうち以下の16種。  
マーケティング結果 `y` を目的変数とした。

|変数|意味|
|:---|:---|
|y|今回のマーケティング結果|
|age|年齢|
|job|職種|
|marital|婚姻状況|
|education|最終学歴|
|default|クレジット支払遅延|
|housing|不動産ローンの有無|
|loan|個人ローンの有無|
|contact|連絡デバイス|
|previous|前回キャンペーン時の接触回数|
|poutcome|前回キャンペーンの結果|
|pdays|前回からの接触からの経過日数|
|emp.var.rate|--|
|cons.price.idx|消費者物価指数|
|cons.conf.idx|消費者信頼感指数|
|euribor3m|3ヵ月EURIBOR|
|nr.employed|四半期ごと就業者数|


#### 前処理
**多重共線性への対処**  
各説明変数はダミー変数に置き換え、 `unknown` や `nonexistent` のように不明な値だけ先に除外しておいた。  
`y` については `yes` だけを使用した。

**年齢の扱い**  
成約が若年層と高齢者層に偏っており、`age` と `y` に比例関係を仮定できないため、  
連続値ではなく10歳ごとに区切ったカテゴリカル変数に置き換えた。  

**以前の接触回数**  
`previous` が0以外の人数が極端に少ないため、接触有無の2値に変換した。

**曜日の扱い**  
よく見ると平日5日しか含まれておらず、平日・休日の違いが現れない。  
ペルソナ分析でもp値が有意でない曜日が多かったため、まとめて除外した。

**結果として、特徴量は61次元となった。**

#### モデルの構築
**リサンプリング**  
学習用とテスト用に分けたうえで、アンダー/オーバーサンプリングを行った。  

1. アンダーサンプリング `imblearn.under_sampling.ClusterCentroids`
2. オーバーサンプリング `imblearn.over_sampling.SMOTE`
3. 両者の混合 `imblearn.combine.SMOTEENN`

の3種類を試したところ、AUCは

1. 0.817
2. 0.859
3. 0.868

最も高かった `SMOTEENN` を採用した。

**ハイパーパラメータのチューニング**  
時間の都合上、すべてデフォルトのまま実施した。

**連続値の標準化・正規化**  
正規化（最小0〜最大1）で試してみたが結果がほぼ変化しなかったため元のデータのまま実施した。

#### 予測モデルの評価

> Confusion matrix(test):  
> [[7159 1806]  
>  [  73 1086]]  
> Accuracy(test) : 0.81440

> precision : 0.3755  
> recall    : 0.9370  
> f-measure : 0.5362

> AUC : 0.868

#### 予測モデルのテストデータへの適用結果
> Confusion matrix(test):  
> [[6803  497]  
>  [ 500  437]]
> Accuracy(test) : 0.87896  

> precision : 0.4679  
> recall    : 0.4664  
> f-measure : 0.4671

> AUC : 0.699